# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [322]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nhl_df_2018=nhl_df[(nhl_df['year']==2018)] 
nhl_df_2018.drop([0,9,18,26],inplace=True) #elimar filas nombre de divicion
cities_1=cities[['Metropolitan area','Population (2016 est.)[8]','NHL']] #estraer filas de importancia
cities_1['team'] = cities_1['NHL'].str.replace(r'(\[[a-z0-9|\s]*\])', '', regex=True) #eliminar los [note 3]
#diccionario cambio de nombre de team elimina las areas y solo deja el nombre el equipo
#con fin de poder hace run merge left 
team={'Tampa Bay Lightning*':'Lightning','Boston Bruins*':'Bruins','Toronto Maple Leafs*':'Maple Leafs',
     'Florida Panthers':'Panthers', 'Detroit Red Wings':'Red Wings', 'Montreal Canadiens':'Canadiens', 
      'Ottawa Senators':'Senators','Buffalo Sabres':'Sabres', 'Washington Capitals*':'Capitals', 
      'Pittsburgh Penguins*':'Penguins','Philadelphia Flyers*':'Flyers', 'Columbus Blue Jackets*':'Blue Jackets',
      'New Jersey Devils*':'Devils','Carolina Hurricanes':'Hurricanes',
      'New York Islanders':'RangersIslandersDevils', 'New York Rangers':'RangersIslandersDevils',
     'Nashville Predators*':'Predators', 'Winnipeg Jets*':'Jets', 'Minnesota Wild*':'Wild', 
      'Colorado Avalanche*':'Avalanche','St. Louis Blues':'Blues', 'Dallas Stars':'Stars', 'Chicago Blackhawks':'Blackhawks',
      'Vegas Golden Knights*':'Golden Knights', 'Anaheim Ducks*':'Ducks', 'San Jose Sharks*':'Sharks', 
      'Los Angeles Kings*':'KingsDucks', 'Calgary Flames':'Flames','Edmonton Oilers':'Oilers', 
      'Vancouver Canucks':'Canucks', 'Arizona Coyotes':'Coyotes'}
nhl_df_2018['team']=nhl_df_2018['team'].map(team) #usa diccionario para mapear y remplazar
#cambio tipo de dato de las columnas
nhl_df_2018['W']=nhl_df_2018['W'].astype(float, errors = 'raise')
nhl_df_2018['L']=nhl_df_2018['L'].astype(float, errors = 'raise')
cities_1['Population (2016 est.)[8]']=cities_1['Population (2016 est.)[8]'].astype(float, errors = 'raise')
#cambio de index para los dos dataframes
nhl_df_2018.set_index('team',inplace=True)
cities_1.set_index('team',inplace=True)
#un merge union left team
union=pd.merge(nhl_df_2018,cities_1, how='left', left_index=True, right_index=True)
#extrae solo las columnas que nos interesan
union_1=union[['W','L','Metropolitan area']]
union_1=union_1.reset_index() #reset el index pasa a ser los anteriores int
union_1=union_1.sort_values('Metropolitan area') #organizar 
#verificamos en union_1 que equipos no tienen metropolitan area y se le adigna una
union_1.loc[union_1.team=='Devils','Metropolitan area']='New York City' 
union_1.loc[union_1.team=='Ducks','Metropolitan area']='Los Angeles'
#agrupamos por area metropolitana y sumamos las victoris y derrotas 
union_2=union_1.groupby(['Metropolitan area']).sum()
#caulcualmos la correlacion entre win/loss
union_2=union_2.assign(ratio = lambda x: (x['W'] /(x['W']+x['L'])))
#extraemos el la ploblaicon de cada area metropolitana
A=union[['Population (2016 est.)[8]','Metropolitan area']]
A.set_index('Metropolitan area',inplace=True)

def nhl_correlation(): 
    #unimos union
    R=pd.merge(union_2,A, how='left', left_index=True, right_index=True)
    R=R.drop_duplicates()
    population_by_region =R['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = R['ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    #return union_1
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr[0]

In [323]:
nhl_correlation()

0.012308996455744249

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [144]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nba_df_2018=nba_df[(nba_df['year']==2018)] 
cities_nba=cities[['Metropolitan area','Population (2016 est.)[8]','NBA']] #estraer filas de importancia
cities_nba['NBA'] = cities_nba['NBA'].str.replace(r'(\[[a-z0-9|\s]*\])', '', regex=True) #eliminar los [note 3]

team={'Toronto Raptors*\xa0(1)':'Raptors', 'Boston Celtics*\xa0(2)':'Celtics', 'Philadelphia 76ers*\xa0(3)':'76ers',
      'Cleveland Cavaliers*\xa0(4)':'Cavaliers', 'Indiana Pacers*\xa0(5)':'Pacers', 'Miami Heat*\xa0(6)':'Heat',
      'Milwaukee Bucks*\xa0(7)':'Bucks', 'Washington Wizards*\xa0(8)':'Wizards', 'Detroit Pistons\xa0(9)':'Pistons',
      'Charlotte Hornets\xa0(10)':'Hornets', 'New York Knicks\xa0(11)':'Knicks', 'Brooklyn Nets\xa0(12)':'Nets',
      'Chicago Bulls\xa0(13)':'Bulls', 'Orlando Magic\xa0(14)':'Magic', 'Atlanta Hawks\xa0(15)':'Hawks', 
      'Houston Rockets*\xa0(1)':'Rockets', 'Golden State Warriors*\xa0(2)':'Warriors', 'Portland Trail Blazers*\xa0(3)':'Trail Blazers', 
      'Oklahoma City Thunder*\xa0(4)':'Thunder', 'Utah Jazz*\xa0(5)':'Jazz', 'New Orleans Pelicans*\xa0(6)':'Pelicans', 
      'San Antonio Spurs*\xa0(7)':'Spurs', 'Minnesota Timberwolves*\xa0(8)':'Timberwolves', 'Denver Nuggets\xa0(9)':'Nuggets',
      'Los Angeles Clippers\xa0(10)':'Clippers', 'Los Angeles Lakers\xa0(11)':'Lakers', 'Sacramento Kings\xa0(12)':'Kings', 
      'Dallas Mavericks\xa0(13)':'Mavericks', 'Memphis Grizzlies\xa0(14)':'Grizzlies', 'Phoenix Suns\xa0(15)':'Suns'}

nba_df_2018['team']=nba_df_2018['team'].map(team) #usa diccionario para mapear y remplazar
#cambio tipo de dato de las columnas
nba_df_2018['W']=nba_df_2018['W'].astype(float, errors = 'raise')
nba_df_2018['L']=nba_df_2018['L'].astype(float, errors = 'raise')
nba_df_2018['W/L%']=nba_df_2018['W/L%'].astype(float, errors = 'raise')
cities_nba['Population (2016 est.)[8]']=cities_nba['Population (2016 est.)[8]'].astype(float, errors = 'raise')
#cambio de index para los dos dataframes
nba_df_2018.set_index('team',inplace=True)
cities_nba.set_index('NBA',inplace=True)
#union de equipos de nba de 2018 y su respectivas areas metropolitanas
union_nba=pd.merge(nba_df_2018,cities_nba, how='left', left_index=True, right_index=True)
#extrae solo las columnas que nos interesan
union_nba_1=union_nba[['W','L','W/L%','Metropolitan area']]
union_nba_1.reset_index(inplace=True) #reset el index pasa a ser los anteriores int
union_nba_1.rename(columns={'index':'team'},inplace=True)
union_nba_1=union_nba_1.sort_values('Metropolitan area') #organizar
#verificamos en union_1 que equipos no tienen metropolitan area y se le adigna una
union_nba_1.loc[union_nba_1.team=='Knicks','Metropolitan area']='New York City' 
union_nba_1.loc[union_nba_1.team=='Nets','Metropolitan area']='New York City'
union_nba_1.loc[union_nba_1.team=='Lakers','Metropolitan area']='Los Angeles'
union_nba_1.loc[union_nba_1.team=='Clippers','Metropolitan area']='Los Angeles'
#agrupamos por area metropolitana y sumamos las victoris y derrotas 
union_nba_2=union_nba_1.groupby(['Metropolitan area']).sum()
#caulcualmos la correlacion entre win/loss
union_nba_2=union_nba_2.assign(ratio = lambda x: (x['W'] /(x['W']+x['L'])))
#extraemos el la ploblaicon de cada area metropolitana
A_nba=cities_nba[['Population (2016 est.)[8]','Metropolitan area']]
A_nba.set_index('Metropolitan area',inplace=True)
def nba_correlation():
    #unimos union
    R=pd.merge(union_nba_2,A_nba, how='left', left_index=True, right_index=True)
    #raise NotImplementedError()
    #return R
    population_by_region =R['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = R['ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    #return 
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    corr=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr[0]

In [145]:
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [71]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

mlb_df_2018=mlb_df[(mlb_df['year']==2018)] 
cities_mlb=cities[['Metropolitan area','Population (2016 est.)[8]','MLB']] #estraer filas de importancia
cities_mlb['MLB'] = cities_mlb['MLB'].str.replace(r'(\[[a-z0-9|\s]*\])', '', regex=True) #eliminar los [note 3]

team={'Boston Red Sox':'Red Sox', 'New York Yankees':'Yankees', 'Tampa Bay Rays':'Rays', 'Toronto Blue Jays':'Blue Jays',
      'Baltimore Orioles':'Orioles','Cleveland Indians':'Indians', 'Minnesota Twins':'Twins', 'Detroit Tigers':'Tigers', 
      'Chicago White Sox':'White Sox', 'Kansas City Royals':'Royals','Houston Astros':'Astros', 'Oakland Athletics':'Athletics',
      'Seattle Mariners':'Mariners', 'Los Angeles Angels':'Angels', 'Texas Rangers':'Rangers', 'Atlanta Braves':'Braves',
      'Washington Nationals':'Nationals', 'Philadelphia Phillies':'Phillies','New York Mets':'Mets','Miami Marlins':'Marlins', 
      'Milwaukee Brewers':'Brewers', 'Chicago Cubs':'Cubs', 'St. Louis Cardinals':'Cardinals', 'Pittsburgh Pirates':'Pirates',
      'Cincinnati Reds':'Reds', 'Los Angeles Dodgers':'Dodgers', 'Colorado Rockies':'Rockies',
      'Arizona Diamondbacks':'Diamondbacks', 'San Francisco Giants':'Giants', 'San Diego Padres':'Padres'}

mlb_df_2018['team']=mlb_df_2018['team'].map(team) #usa diccionario para mapear y remplazar
#cambio tipo de dato de las columnas
mlb_df_2018['W']=mlb_df_2018['W'].astype(float, errors = 'raise')
mlb_df_2018['L']=mlb_df_2018['L'].astype(float, errors = 'raise')
cities_mlb['Population (2016 est.)[8]']=cities_mlb['Population (2016 est.)[8]'].astype(float, errors = 'raise')
#cambio de index para los dos dataframes
mlb_df_2018.set_index('team',inplace=True)
cities_mlb.set_index('MLB',inplace=True)
#union de equipos de mlb de 2018 y su respectivas areas metropolitanas
union_mlb=pd.merge(mlb_df_2018,cities_mlb, how='left', left_index=True, right_index=True)
#extrae solo las columnas que nos interesan
union_mlb_1=union_mlb[['W','L','W-L%','Metropolitan area']]
union_mlb_1.reset_index(inplace=True) #reset el index pasa a ser los anteriores int
union_mlb_1.rename(columns={'index':'team'},inplace=True)
union_mlb_1=union_mlb_1.sort_values('Metropolitan area') #organizar
#verificamos en union_1 que equipos no tienen metropolitan area y se le adigna una
cambios_area={'Giants':'San Francisco Bay Area','Athletics':'San Francisco Bay Area',
             'Cubs':'Chicago','White Sox':'Chicago','Yankees':'New York City', 'Mets':'New York City',
             'Angels':'Los Angeles','Dodgers':'Los Angeles'}
for clave, valor in cambios_area.items():
    union_mlb_1.loc[union_mlb_1.team==clave,'Metropolitan area']=valor
#agrupamos por area metropolitana y sumamos las victoris y derrotas 
union_mlb_2=union_mlb_1.groupby(['Metropolitan area']).sum()
#caulcualmos la correlacion entre win/loss
union_mlb_2=union_mlb_2.assign(ratio = lambda x: (x['W'] /(x['W']+x['L'])))
#extraemos el la ploblaicon de cada area metropolitana
A_mlb=cities_mlb[['Population (2016 est.)[8]','Metropolitan area']]
A_mlb.set_index('Metropolitan area',inplace=True)


def mlb_correlation(): 
    R=pd.merge(union_mlb_2,A_mlb, how='left', left_index=True, right_index=True)
    #raise NotImplementedError()
    
    population_by_region = R['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region =R['ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr_mlb=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr_mlb[0]

In [72]:
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [75]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nfl_df_2018=nfl_df[(nfl_df['year']==2018)] 
nfl_df_2018.drop([0,5,10,15,20,25,30,35],inplace=True) #elimar filas nombre de divicion
cities_nfl=cities[['Metropolitan area','Population (2016 est.)[8]','NFL']] #estraer filas de importancia
cities_nfl['NFL'] = cities_nfl['NFL'].str.replace(r'(\[[a-z0-9|\s]*\])', '', regex=True) #eliminar los [note 3]
team={'New England Patriots*':'Patriots', 'Miami Dolphins':'Dolphins', 'Buffalo Bills':'Bills', 'New York Jets':'Jets',
      'Baltimore Ravens*':'Ravens', 'Pittsburgh Steelers':'Steelers', 'Cleveland Browns':'Browns', 'Cincinnati Bengals':'Bengals',
      'Houston Texans*':'Texans', 'Indianapolis Colts+':'Colts', 'Tennessee Titans':'Titans', 'Jacksonville Jaguars':'Jaguars', 
      'Kansas City Chiefs*':'Chiefs', 'Los Angeles Chargers+':'Chargers', 'Denver Broncos':'Broncos', 'Oakland Raiders':'Raiders',
      'Dallas Cowboys*':'Cowboys', 'Philadelphia Eagles+':'Eagles', 'Washington Redskins':'Redskins', 'New York Giants':'Giants',
      'Chicago Bears*':'Bears', 'Minnesota Vikings':'Vikings', 'Green Bay Packers':'Packers', 'Detroit Lions':'Lions',
      'New Orleans Saints*':'Saints','Carolina Panthers':'Panthers', 'Atlanta Falcons':'Falcons', 
      'Tampa Bay Buccaneers':'Buccaneers', 'Los Angeles Rams*':'Rams',
      'Seattle Seahawks+':'Seahawks', 'San Francisco 49ers':'49ers', 'Arizona Cardinals':'Cardinals'}

nfl_df_2018['team']=nfl_df_2018['team'].map(team) #usa diccionario para mapear y remplazar
#cambio tipo de dato de las columnas
nfl_df_2018['W']=nfl_df_2018['W'].astype(float, errors = 'raise')
nfl_df_2018['L']=nfl_df_2018['L'].astype(float, errors = 'raise')
cities_nfl['Population (2016 est.)[8]']=cities_nfl['Population (2016 est.)[8]'].astype(float, errors = 'raise')
#cambio de index para los dos dataframes
nfl_df_2018.set_index('team',inplace=True)
cities_nfl.set_index('NFL',inplace=True)
#union de equipos de nfl de 2018 y su respectivas areas metropolitanas
union_nfl=pd.merge(nfl_df_2018,cities_nfl, how='left', left_index=True, right_index=True)
#extrae solo las columnas que nos interesan
union_nfl_1=union_nfl[['W','L','W-L%','Metropolitan area']]
union_nfl_1.reset_index(inplace=True) #reset el index pasa a ser los anteriores int
union_nfl_1.rename(columns={'index':'team'},inplace=True)
union_nfl_1=union_nfl_1.sort_values('Metropolitan area') #organizar
#verificamos en union_1 que equipos no tienen metropolitan area y se le adigna una
cambios_area={'49ers':'San Francisco Bay Area','Raiders':'San Francisco Bay Area',
             'Giants':'New York City', 'Jets':'New York City',
             'Chargers':'Los Angeles','Rams':'Los Angeles'}
for clave, valor in cambios_area.items():
    union_nfl_1.loc[union_nfl_1.team==clave,'Metropolitan area']=valor
#agrupamos por area metropolitana y sumamos las victoris y derrotas 
union_nfl_2=union_nfl_1.groupby(['Metropolitan area']).sum()
#caulcualmos la correlacion entre win/loss
union_nfl_2=union_nfl_2.assign(ratio = lambda x: (x['W'] /(x['W']+x['L'])))
#extraemos el la ploblaicon de cada area metropolitana
A_nfl=cities_nfl[['Population (2016 est.)[8]','Metropolitan area']]
A_nfl.set_index('Metropolitan area',inplace=True)
def nfl_correlation(): 
    R=pd.merge(union_nfl_2,A_nfl, how='left', left_index=True, right_index=True)
    #raise NotImplementedError()
    
    population_by_region = R['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = R['ratio']# pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr_nfl= stats.pearsonr(population_by_region, win_loss_by_region)
    return corr_nfl[0]

In [76]:
nfl_correlation()

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values